In [1]:


# from data_multi import Bandit_multi
# from learner_diag import NeuralUCBDiag
import numpy as np
import argparse
import pickle
import os
import time
import torch
import numpy as np
from scipy.stats import truncnorm
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
device
# torch.set_num_threads(8)
# torch.set_num_interop_threads(8)



def truncated_gaussian(mean, variance, a, b, size):
    # Calculate the standard deviation from the variance
    std_dev = np.sqrt(variance)

    # Calculate the lower and upper bounds for truncation
    lower_bound = (a - mean) / std_dev
    upper_bound = (b - mean) / std_dev

    # Generate samples from the truncated normal distribution
    samples = truncnorm.rvs(lower_bound, upper_bound, loc=mean, scale=std_dev, size=size)

    return np.array(samples)




class Evaluation:

    def __init__(self, game_name, task, n_folds, horizon, game, label, context_type):
        self.game_name = game_name
        self.task = task
        self.n_folds = n_folds
        self.horizon = horizon
        self.game = game
        self.label =  label
        self.context_type = context_type
        

    def get_outcomes(self, game, ):
        outcomes = np.random.choice( game.n_outcomes , p= list( game.outcome_dist.values() ), size= self.horizon) 
        return outcomes

    def get_feedback(self, game, action, outcome):
        return game.FeedbackMatrix[ action ][ outcome ]

    def eval_policy_once(self, alg, game, job):

        alg.reset()

        context_generator, jobid = job

        np.random.seed(jobid)

        cumRegret =  np.zeros(self.horizon, dtype =float)
        compteur = None
        for t in range(self.horizon):
            compteur = t
            if t % 1000 == 0 :
                print(t)

            context,distribution = context_generator.get_context()
            outcome = np.random.choice( 2 , p = distribution )

            context = context 
            action = alg.get_action(t, context)

            print('t', t, 'action', action, 'outcome', outcome, 'gaps', game.LossMatrix[0,...] @ distribution - game.LossMatrix[1,...] @ distribution  )
            print()
            feedback =  self.get_feedback( game, action, outcome )

            alg.update(action, feedback, outcome, t, context )

            i_star = np.argmin(  [ game.LossMatrix[i,...] @ np.array( distribution ) for i in range(alg.N) ]  )
            loss_diff = game.LossMatrix[action,...] - game.LossMatrix[i_star,...]
            val = loss_diff @ np.array( distribution )
            cumRegret[t] =  val

        return cumRegret
    

class LinearContexts:
    def __init__(self,  w, task):
        self.d = len(w) # number of features
        self.w = w
        self.type = 'linear'
        self.task = task
    
    def get_context(self, ):

        if self.task == 'imbalanced':
            context = truncated_gaussian(0, 0.1, 0, 1, self.d) if np.random.uniform(0,1)<0.5 else truncated_gaussian(1, 0.1, 0, 1, self.d)
        else:
            context = truncated_gaussian(0.5, 1, 0, 1, self.d)
        cont = context.reshape(self.d,1)
        val = self.w @ cont
        val = [ val[0], 1-val[0] ]
        return cont - np.ones((5,1)) * 0.5 , val 
    
class ToyContexts:

    def __init__(self, ):
        self.type = 'toy'
        self.d_context = 2

    def get_context(self, ):
        p = 1 if np.random.uniform(0, 1)>0.5 else 0 
        return np.array([1,p]).reshape(2,1), [p, 1-p]



In [2]:
import neuralcbpside_v1
import neuralcbpside_v2
import games
import synthetic_data
import numpy as np
import geometry_v3
import cbpside

horizon = 1000
lbd = 1
game = games.apple_tasting( )

size = 5
w = np.array([1/size]*size)
context_generator = LinearContexts( w,'imbalanced' )

# w = np.array([0,1])
# context_generator = ToyContexts()

alg = cbpside.CBPside( game, len(w), 1.01, lbd )
eval1 = Evaluation(None, None, None, horizon, game, None, None)
job1 = context_generator, 0
cumRegret1 = eval1.eval_policy_once(alg, game, job1)

Restricted license - for non-production use only - expires 2024-10-28
0
t 0 action 0 outcome 0 gaps 0.6527247375416398

Yit shape (1, 1)
t 1 action 1 outcome 1 gaps 0.6992744976791685

Yit shape (2, 1)
factor 1.8325546111576978 width [[0.61033684]]
factor 5.665109222315396 width [[0.61407321]]
estimate [array([[-0.25966689]]), array([[-0.24533504],
       [ 0.        ]])]
pair [0, 1] tdelta [0.24533504] confidence [[3.47879181]]
union1 [0 1]
t 2 action 1 outcome 1 gaps -0.46390234929033314

Yit shape (2, 2)
factor 2.048147073968205 width [[0.53859264]]
factor 6.09629414793641 width [[0.48629411]]
estimate [array([[0.22331491]]), array([[-0.04897481],
       [ 0.        ]])]
pair [0, 1] tdelta [0.04897481] confidence [[2.96459194]]
union1 [0 1]
t 3 action 1 outcome 1 gaps 0.43142415549870305

Yit shape (2, 3)
factor 2.177410022515475 width [[0.53896754]]
factor 6.35482004503095 width [[0.49705189]]
estimate [array([[0.29065215]]), array([[0.2785098],
       [0.       ]])]
pair [0, 1] td